In [ ]:
import sys
import os
import matplotlib.pyplot as plt

# Assuming the package is one directory above the current working directory
parent_dir = os.path.abspath('../..')
sys.path.append(parent_dir)

import numpy as np
import casadi as cs

from UVarm import robotdynamics, pluck, urdfp
import spatial_casadi as sc

In [2]:
root = "base_link"
tip = "alpha_standard_jaws_base_link"

path_to_urdf = f"{parent_dir}/resources/urdf/alpha_5_robot.urdf"
rig_dyn = robotdynamics(path_to_urdf, root, tip)

ss = rig_dyn.ssyms

jit after {'jit': True, 'jit_options': {'flags': '-Ofast'}, 'compiler': 'shell'}
number of joints = 4


In [ ]:
# using plucker basis :=[w, v]

# self.u_min = np.array([-2.83664, -0.629139, -0.518764, -0.54])
# self.u_max = np.array([2.83664, 0.629139, 0.518764, 0.54])
joint_min = np.array([0.0, 1.5, 0.05, 0.0])
joint_max = np.array([5.7, 3.4, 3.4, 5.7]) 

rho = 1 #kg/L

mc = cs.SX.sym('mc', 7)

mc_v = [7e-6, 0.032, 1716e-6, 0.017, 0.201, 2443e-6, 0.226]

M_A_0 = cs.vertcat(mc[0]*rho, mc[0]*rho, 0, mc[1]*rho, mc[1]*rho, mc[3]*rho)
M_A_1 = cs.vertcat(0, mc[2]*rho, mc[2]*rho, mc[3]*rho, mc[4]*rho, mc[4]*rho)
M_A_2 = cs.vertcat(mc[0]*rho, 0, mc[0]*rho, mc[1]*rho, mc[3]*rho, mc[1]*rho)
M_A_3 = cs.vertcat(mc[5]*rho, mc[5]*rho, 0, mc[6]*rho, mc[6]*rho, mc[3]*rho)
MA__cof = cs.vertcat(M_A_0, M_A_1, M_A_2, M_A_3)

In [4]:
D_u_0 = cs.DM([0, 0, 0, 0, 0, 0])
D_u_1 = cs.DM([0, 0, 0, 0, 0, 0])
D_u_2 = cs.DM([0, 0, 0, 0, 0, 0])
D_u_3 = cs.DM([0, 0, 0, 0, 0, 0])
Du__cof = cs.vertcat(D_u_0, D_u_1, D_u_2, D_u_3)

In [5]:
du = cs.SX.sym('du', 4)

du_v = [0.26, 0.3, 1.6, 1.8]

D_uu_0 = cs.vertcat(0, 0, 0, du[0]*rho, du[0]*rho, du[1]*rho)
D_uu_1 = cs.vertcat(0, 0, 0, du[1]*rho, du[2]*rho, du[2]*rho)
D_uu_2 = cs.vertcat(0, 0, 0, du[0]*rho, du[1]*rho, du[0]*rho)
D_uu_3 = cs.vertcat(0, 0, 0, du[3]*rho, du[3]*rho, du[1]*rho)
Duu__cof = cs.vertcat(D_uu_0, D_uu_1, D_uu_2, D_uu_3)

In [6]:
COB_m0 = cs.DM([-1e-03, -2e-03, -32e-03])
COB_m1 = cs.DM([73e-3, 0, -2e-3])
COB_m2 = cs.DM([3e-3, 1e-3, -17e-3])
COB_m3 = cs.DM([0e-3, 3e-3, -98e-3])
COB__m = cs.vertcat(COB_m0, COB_m1, COB_m2, COB_m3)

volume__ = cs.vertcat(1.8e-5, 0.000203, 2.5e-5, 0.000155) #m3

In [7]:
p0_Hyd__sx = cs.vertcat(MA__cof, Du__cof, Duu__cof, volume__ ,COB__m, rho)
p0_Hyd__fun = cs.Function('p0Hydsx', [mc, du],[p0_Hyd__sx])
arm_Hyd__p_est = p0_Hyd__fun(mc_v, du_v)
arm_Hyd__p_est

DM([7e-06, 7e-06, 0, 0.032, 0.032, 0.017, 0, 0.001716, 0.001716, 0.017, 0.201, 0.201, 7e-06, 0, 7e-06, 0.032, 0.017, 0.032, 0.002443, 0.002443, 0, 0.226, 0.226, 0.017, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.26, 0.26, 0.3, 0, 0, 0, 0.3, 1.6, 1.6, 0, 0, 0, 0.26, 0.3, 0.26, 0, 0, 0, 1.8, 1.8, 0.3, 1.8e-05, 0.000203, 2.5e-05, 0.000155, -0.001, -0.002, -0.032, 0.073, 0, -0.002, 0.003, 0.001, -0.017, 0, 0.003, -0.098, 1])

In [ ]:
# reducing model parameters by assuming non rotating axis are inertialess or inactivity
Gear_p = cs.vertcat(2253.54, 2253.54, 2253.54, 340.4)
rigid_p0 = cs.vertcat(0.0, 0.0, 0.0, 1e-05, 0.0, 0.0,
                           0.0, 0.0, 0.0, 0.0, 1e-05, 0.0,
                           0.0, 0.0, 1e-05, 0.0,
                           1e-05, 
                           0.0, 0.0, 0.0, 0.0, 
                           3.0, 2.3, 2.2, 0.3, 
                           0.0, 0.0, 0.0, 0.0,
                           3.0, 1.8, 1.0, 1.15)

rigid_body_p_est = cs.vertcat(Gear_p, rigid_p0)

trivial_Ir0 = [0, 0, 0 , 0, 0, 0, 0]

v_c0 = [0, 0, 0 , 0, 0, 0]